In [ ]:

server = { 
    "ip_pool": ["192.168.1.100", "192.168.1.101", "192.168.1.102"], 
    "leases": {} 
} 
 
client = { 
    "mac": "AA:BB:CC:DD:EE:FF", 
    "ip": None 
} 
 
def send_discover(): 
    print("\n[CLIENT] Step 1: Sending DHCP DISCOVER") 
    return { 
        "type": "DISCOVER", 
        "mac": client["mac"] 
    } 
 
def make_offer(discover): 
    print("\n[SERVER] Step 2: Making DHCP OFFER") 
    if not server["ip_pool"]: 
        print("No IPs available!") 
        return None 
     
    offered_ip = server["ip_pool"].pop(0) 
    return { 
        "type": "OFFER", 
        "mac": discover["mac"], 
        "ip": offered_ip 
    } 
 
def send_request(offer): 
    print("\n[CLIENT] Step 3: Sending DHCP REQUEST") 
    return { 
        "type": "REQUEST", 
        "mac": offer["mac"], 
        "ip": offer["ip"] 
    } 
 
def send_ack(request): 
    print("\n[SERVER] Step 4: Sending DHCP ACK") 
    server["leases"][request["mac"]] = request["ip"] 
    return { 
        "type": "ACK", 
        "mac": request["mac"], 
        "ip": request["ip"] 
    } 
 
def main(): 
    print("=== Simple DHCP Simulation ===") 
     
    discover = send_discover() 
     
     
    offer = make_offer(discover) 
    if not offer: 
        return 
     
    request = send_request(offer) 
     
    ack = send_ack(request) 
     
    client["ip"] = ack["ip"] 
     
    print("\n=== Result ===") 
    print(f"Client {client['mac']} got IP: {client['ip']}") 
    print("Server leases:", server["leases"]) 
 
if __name__ == "__main__": 
    main()

=== Simple DHCP Simulation ===

[CLIENT] Step 1: Sending DHCP DISCOVER

[SERVER] Step 2: Making DHCP OFFER

[CLIENT] Step 3: Sending DHCP REQUEST

[SERVER] Step 4: Sending DHCP ACK

=== Result ===
Client AA:BB:CC:DD:EE:FF got IP: 192.168.1.100
Server leases: {'AA:BB:CC:DD:EE:FF': '192.168.1.100'}


In [ ]:
import socket
import json

server = { 
    "ip_pool": ["192.168.1.100", "192.168.1.101", "192.168.1.102"], 
    "leases": {} 
} 
 
def client(connection):
    data = connection.recv(1024).decode()
    if not data:
        return
    
    request=json.loads(data)
    
    if request["type"] == "DISCOVER":
        print("\nReceived DHCP DISCOVER from", request["mac"])

        if not server["ip_pool"]:
            print("No available IPs!")
            connection.send(json.dumps({"type": "NAK"}).encode())
            return
        
        offer_ip = server["ip_pool"].pop(0)
        offer = {"type": "OFFER", 
                 "mac": request["mac"], 
                 "ip": offer_ip}
        print(f"Offering IP {offer_ip} to {request['mac']}")
        connection.send(json.dumps(offer).encode())
        
        request_data = connection.recv(1024).decode()
        request = json.loads(request_data)

        if request["type"]== "REQUEST":
            print(f"request from {request['mac']} for {request['ip']}")

            server["leases"][request["mac"]] = request["ip"]
            ack = {"type": "ACK",
                   "mac": request["mac"],
                   "ip" : request["ip"]}
            
            print(f"ip address: {request["ip"]} goes to {request["mac"]}")
            connection.send(json.dumps(ack).encode())
    connection.close()
def main():
    print("server is running..")
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(("0.0.0.0", 6000))
    server_socket.listen(3)

    while True:
        connection, addr = server_socket.accept()
        print(f"\n Connection from {addr}")
        client(connection)


if __name__ == "__main__":
    main()




server is running..

 Connection from ('127.0.0.1', 53112)

Received DHCP DISCOVER from AA:BB:CC:DD:EE:FF
Offering IP 192.168.1.100 to AA:BB:CC:DD:EE:FF
request from AA:BB:CC:DD:EE:FF for 192.168.1.100
ip address: 192.168.1.100 goes to AA:BB:CC:DD:EE:FF

 Connection from ('127.0.0.1', 53114)

Received DHCP DISCOVER from AA:BB:CC:DD:EE:FF
Offering IP 192.168.1.101 to AA:BB:CC:DD:EE:FF
request from AA:BB:CC:DD:EE:FF for 192.168.1.101
ip address: 192.168.1.101 goes to AA:BB:CC:DD:EE:FF

 Connection from ('127.0.0.1', 53116)

Received DHCP DISCOVER from AA:BB:CC:DD:EE:FF
Offering IP 192.168.1.102 to AA:BB:CC:DD:EE:FF
request from AA:BB:CC:DD:EE:FF for 192.168.1.102
ip address: 192.168.1.102 goes to AA:BB:CC:DD:EE:FF
